.

q1_softmax.py codes:

In [1]:
import numpy as np
import tensorflow as tf

def softmax(x):
    """
  Compute the softmax function in tensorflow.

  You might find the tensorflow functions tf.exp, tf.reduce_max,
  tf.reduce_sum, tf.expand_dims useful. (Many solutions are possible, so you may
  not need to use all of these functions). Recall also that many common
  tensorflow operations are sugared (e.g. x * y does a tensor multiplication
  if x and y are both tensors). Make sure to implement the numerical stability
  fixes as in the previous homework!

    Args:
    x:   tf.Tensor with shape (n_samples, n_features). Note feature vectors are
         represented by row-vectors. (For simplicity, no need to handle 1-d
         input as in the previous homework)
    Returns:
    out: tf.Tensor with shape (n_sample, n_features). You need to construct this
         tensor in this problem.
    """

    ### YOUR CODE HERE
    row=tf.shape(x)[0]
    x_max=tf.reduce_max(x,axis=1)
    x=tf.exp(x-tf.reshape(x_max,[row,1]))
    x_sum=tf.reshape(tf.reduce_sum(x,axis=1),[row,1])
    out=x/x_sum
    ### END YOUR CODE
      
    return out 

def cross_entropy_loss(y, yhat):
  """
  Compute the cross entropy loss in tensorflow.

  y is a one-hot tensor of shape (n_samples, n_classes) and yhat is a tensor
  of shape (n_samples, n_classes). y should be of dtype tf.int32, and yhat should
  be of dtype tf.float32.

  The functions tf.to_float, tf.reduce_sum, and tf.log might prove useful. (Many
  solutions are possible, so you may not need to use all of these functions).

  Note: You are NOT allowed to use the tensorflow built-in cross-entropy
        functions.

  Args:
    y:    tf.Tensor with shape (n_samples, n_classes). One-hot encoded.
    yhat: tf.Tensorwith shape (n_sample, n_classes). Each row encodes a
          probability distribution and should sum to 1.
  Returns:
    out:  tf.Tensor with shape (1,) (Scalar output). You need to construct this
          tensor in the problem.
  """
  ### YOUR CODE HERE
  out=-tf.reduce_sum(tf.matmul(tf.to_float(y),tf.log(yhat), transpose_b=True),axis=1)
  ### END YOUR CODE
  return out

.

softmax tests:

In [2]:
  test1 = softmax(tf.convert_to_tensor(
      np.array([[1001,1002],[3,4]]), dtype=tf.float32))
  with tf.Session():
      test1 = test1.eval()
  assert np.amax(np.fabs(test1 - np.array(
      [0.26894142,  0.73105858]))) <= 1e-6
  np.amax(np.fabs(test1 - np.array(
      [0.26894142,  0.73105858]))) <= 1e-6
    
  test2 = softmax(tf.convert_to_tensor(
      np.array([[-1001,-1002]]), dtype=tf.float32))
  with tf.Session():
      test2 = test2.eval()
  assert np.amax(np.fabs(test2 - np.array(
      [0.73105858, 0.26894142]))) <= 1e-6
  np.amax(np.fabs(test1 - np.array(
      [0.26894142,  0.73105858]))) <= 1e-6
  print("Basic (non-exhaustive) softmax tests pass\n")

Basic (non-exhaustive) softmax tests pass



cross_entropy_loss test:

In [3]:
  y = np.array([[0, 1], [1, 0], [1, 0]])
  yhat = np.array([[.5, .5], [.5, .5], [.5, .5]])

  test1 = cross_entropy_loss(
      tf.convert_to_tensor(y, dtype=tf.int32),
      tf.convert_to_tensor(yhat, dtype=tf.float32))
  with tf.Session():
    test1 = test1.eval()
  result = -3 * np.log(.5)
  assert np.amax(np.fabs(test1 - result)) <= 1e-6
  print("Basic (non-exhaustive) cross-entropy tests pass\n")
  #print(test1)

Basic (non-exhaustive) cross-entropy tests pass



model.py codes:

In [4]:
class Model(object):
  """Abstracts a Tensorflow graph for a learning task.

  We use various Model classes as usual abstractions to encapsulate tensorflow
  computational graphs. Each algorithm you will construct in this homework will
  inherit from a Model object.
  """

  def load_data(self):
    """Loads data from disk and stores it in memory.

    Feel free to add instance variables to Model object that store loaded data.    
    """
    raise NotImplementedError("Each Model must re-implement this method.")

  def add_placeholders(self):
    """Adds placeholder variables to tensorflow computational graph.

    Tensorflow uses placeholder variables to represent locations in a
    computational graph where data is inserted.  These placeholders are used as
    inputs by the rest of the model building code and will be fed data during
    training.

    See for more information:

    https://www.tensorflow.org/versions/r0.7/api_docs/python/io_ops.html#placeholders
    """
    raise NotImplementedError("Each Model must re-implement this method.")

  def create_feed_dict(self, input_batch, label_batch):
    """Creates the feed_dict for training the given step.

    A feed_dict takes the form of:

    feed_dict = {
        <placeholder>: <tensor of values to be passed for placeholder>,
        ....
    }
  
    If label_batch is None, then no labels are added to feed_dict.

    Hint: The keys for the feed_dict should be a subset of the placeholder
          tensors created in add_placeholders.
    
    Args:
      input_batch: A batch of input data.
      label_batch: A batch of label data.
    Returns:
      feed_dict: The feed dictionary mapping from placeholders to values.
    """
    raise NotImplementedError("Each Model must re-implement this method.")

  def add_model(self, input_data):
    """Implements core of model that transforms input_data into predictions.

    The core transformation for this model which transforms a batch of input
    data into a batch of predictions.

    Args:
      input_data: A tensor of shape (batch_size, n_features).
    Returns:
      out: A tensor of shape (batch_size, n_classes)
    """
    raise NotImplementedError("Each Model must re-implement this method.")

  def add_loss_op(self, pred):
    """Adds ops for loss to the computational graph.

    Args:
      pred: A tensor of shape (batch_size, n_classes)
    Returns:
      loss: A 0-d tensor (scalar) output
    """
    raise NotImplementedError("Each Model must re-implement this method.")

  def run_epoch(self, sess, input_data, input_labels):
    """Runs an epoch of training.

    Trains the model for one-epoch.
  
    Args:
      sess: tf.Session() object
      input_data: np.ndarray of shape (n_samples, n_features)
      input_labels: np.ndarray of shape (n_samples, n_classes)
    Returns:
      average_loss: scalar. Average minibatch loss of model on epoch.
    """
    raise NotImplementedError("Each Model must re-implement this method.")

  def fit(self, sess, input_data, input_labels):
    """Fit model on provided data.

    Args:
      sess: tf.Session()
      input_data: np.ndarray of shape (n_samples, n_features)
      input_labels: np.ndarray of shape (n_samples, n_classes)
    Returns:
      losses: list of loss per epoch
    """
    raise NotImplementedError("Each Model must re-implement this method.")

  def predict(self, sess, input_data, input_labels=None):
    """Make predictions from the provided model.
    Args:
      sess: tf.Session()
      input_data: np.ndarray of shape (n_samples, n_features)
      input_labels: np.ndarray of shape (n_samples, n_classes)
    Returns:
      average_loss: Average loss of model.
      predictions: Predictions of model on input_data
    """
    raise NotImplementedError("Each Model must re-implement this method.")

class LanguageModel(Model):
  """Abstracts a Tensorflow graph for learning language models.

  Adds ability to do embedding.
  """

  def add_embedding(self):
    """Add embedding layer. that maps from vocabulary to vectors.
    """
    raise NotImplementedError("Each Model must re-implement this method.")


'

q1_clasifier.py code:

In [5]:
import time
import math
import numpy as np
import tensorflow as tf
#from q1_softmax import softmax
#from q1_softmax import cross_entropy_loss
from model import Model
from utils import data_iterator

class Config(object):
  """Holds model hyperparams and data information.

  The config class is used to store various hyperparameters and dataset
  information parameters. Model objects are passed a Config() object at
  instantiation.
  """
  batch_size = 64
  n_samples = 1024
  n_features = 100
  n_classes = 5
  # You may adjust the max_epochs to ensure convergence.
  max_epochs = 100
  # You may adjust this learning rate to ensure convergence.
  lr = 1e-4 

In [6]:
class SoftmaxModel(Model):
  """Implements a Softmax classifier with cross-entropy loss."""

  def load_data(self):
    """Creates a synthetic dataset and stores it in memory."""
    np.random.seed(1234)
    self.input_data = np.random.rand(
        self.config.n_samples, self.config.n_features)
    self.input_labels = np.ones((self.config.n_samples,), dtype=np.int32)

  def add_placeholders(self):
    """Generate placeholder variables to represent the input tensors.

    These placeholders are used as inputs by the rest of the model building
    code and will be fed data during training.

    Adds following nodes to the computational graph

    input_placeholder: Input placeholder tensor of shape
                       (batch_size, n_features), type tf.float32
    labels_placeholder: Labels placeholder tensor of shape
                       (batch_size, n_classes), type tf.int32

    Add these placeholders to self as the instance variables
  
      self.input_placeholder
      self.labels_placeholder

    (Don't change the variable names)
    """
    ### YOUR CODE HERE
    #cfg=Config()
    self.input_placeholder=tf.placeholder(tf.float32,[self.config.batch_size,self.config.n_features],name="input_placeholder")
    self.labels_placeholder=tf.placeholder(tf.int32,[self.config.batch_size,self.config.n_classes],name="labels_placeholder")
    ### END YOUR CODE

  def create_feed_dict(self, input_batch, label_batch):
    """Creates the feed_dict for softmax classifier.

    A feed_dict takes the form of:

    feed_dict = {
        <placeholder>: <tensor of values to be passed for placeholder>,
        ....
    }

    If label_batch is None, then no labels are added to feed_dict.

    Hint: The keys for the feed_dict should match the placeholder tensors
          created in add_placeholders.
    
    Args:
      input_batch: A batch of input data.
      label_batch: A batch of label data.
    Returns:
      feed_dict: The feed dictionary mapping from placeholders to values.
    """
    ### YOUR CODE HERE
    feed_dict= {self.input_placeholder:input_batch, self.labels_placeholder:label_batch}
    ### END YOUR CODE
    return feed_dict

  def add_training_op(self, loss):
    """Sets up the training Ops.

    Creates an optimizer and applies the gradients to all trainable variables.
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train. See 

    https://www.tensorflow.org/versions/r0.7/api_docs/python/train.html#Optimizer

    for more information.

    Hint: Use tf.train.GradientDescentOptimizer to get an optimizer object.
          Calling optimizer.minimize() will return a train_op object.

    Args:
      loss: Loss tensor, from cross_entropy_loss.
    Returns:
      train_op: The Op for training.
    """
    ### YOUR CODE HERE
    #raise NotImplementedError
    #print("add_training_op NotImplemented")
    optimizer=tf.train.GradientDescentOptimizer(self.config.lr)
    train_op=optimizer.minimize(loss)
    ### END YOUR CODE
    return train_op

  def add_model(self, input_data):
    """Adds a linear-layer plus a softmax transformation

    The core transformation for this model which transforms a batch of input
    data into a batch of predictions. In this case, the mathematical
    transformation effected is

    y = softmax(xW + b)

    Hint: Make sure to create tf.Variables as needed. Also, make sure to use
          tf.name_scope to ensure that your name spaces are clean.
    Hint: For this simple use-case, it's sufficient to initialize both weights W
          and biases b with zeros.

    Args:
      input_data: A tensor of shape (batch_size, n_features).
    Returns:
      out: A tensor of shape (batch_size, n_classes)
    """
    tf.name_scope("Softmax")
    ### YOUR CODE HERE
    #if hasattr(self, 'weights') and self.weights is not none:
    #    weights = self.weights
    #else:
    weights=tf.Variable(tf.truncated_normal([self.config.n_classes,self.config.n_features],
                                            stddev=1.0 / math.sqrt(self.config.n_features)))
    #if hasattr(self, 'biases') and self.weights is not none:
    #    biases = self.biases
    #else:
    biases=tf.Variable(tf.zeros(shape=[self.config.n_classes]))
        
    
    
    #h=tf.Variable(tf.zeros(shape=[self.config.batch_size,self.config.n_classes]))
    
    #print("add_model NotImplemented")
    #y=softmax(tf.matmul(weights,input_data,transpose_b=True)+biases)
    #h=tf.matmul(weights,input_data,transpose_b=True)
    h=tf.matmul(input_data,weights,transpose_b=True) + biases
    h=softmax(h)
    out=h
    ### END YOUR CODE
    return out

  def add_loss_op(self, pred):
    """Adds cross_entropy_loss ops to the computational graph.

    Hint: Use the cross_entropy_loss function we defined. This should be a very
          short function.
    Args:
      pred: A tensor of shape (batch_size, n_classes)
    Returns:
      loss: A 0-d tensor (scalar)
    """
    ### YOUR CODE HERE
    #print("add_loss_op NotImplemented")
    loss=tf.reduce_sum(cross_entropy_loss(self.labels_placeholder,pred))
    ### END YOUR CODE
    return loss

  def run_epoch(self, sess, input_data, input_labels):
    """Runs an epoch of training.

    Trains the model for one-epoch.
  
    Args:
      sess: tf.Session() object
      input_data: np.ndarray of shape (n_samples, n_features)
      input_labels: np.ndarray of shape (n_samples, n_classes)
    Returns:
      average_loss: scalar. Average minibatch loss of model on epoch.
    """
    # And then after everything is built, start the training loop.
    average_loss = 0
    for step, (input_batch, label_batch) in enumerate(
        data_iterator(input_data, input_labels,
                      batch_size=self.config.batch_size,
                      label_size=self.config.n_classes)):

      # Fill a feed dictionary with the actual set of images and labels
      # for this particular training step.
      feed_dict = self.create_feed_dict(input_batch, label_batch)

      # Run one step of the model.  The return values are the activations
      # from the `self.train_op` (which is discarded) and the `loss` Op.  To
      # inspect the values of your Ops or variables, you may include them
      # in the list passed to sess.run() and the value tensors will be
      # returned in the tuple from the call.
      _, loss_value = sess.run([self.train_op, self.loss], feed_dict=feed_dict)
      average_loss += loss_value

    average_loss = average_loss / step
    return average_loss 

  def fit(self, sess, input_data, input_labels):
    """Fit model on provided data.

    Args:
      sess: tf.Session()
      input_data: np.ndarray of shape (n_samples, n_features)
      input_labels: np.ndarray of shape (n_samples, n_classes)
    Returns:
      losses: list of loss per epoch
    """
    losses = []
    for epoch in range(self.config.max_epochs):
      start_time = time.time()
      average_loss = self.run_epoch(sess, input_data, input_labels)
      duration = time.time() - start_time
      # Print status to stdout.
      print(('Epoch %d: loss = %.2f (%.3f sec)'
             % (epoch, average_loss, duration)))
      losses.append(average_loss)
    return losses

  def __init__(self, config):
    """Initializes the model.

    Args:
      config: A model configuration object of type Config
    """
    self.config = config
    # Generate placeholders for the images and labels.
    self.load_data()
    self.add_placeholders()
    self.pred = self.add_model(self.input_placeholder)
    self.loss = self.add_loss_op(self.pred)
    self.train_op = self.add_training_op(self.loss)
  
def test_SoftmaxModel():
  """Train softmax model for a number of steps."""
  config = Config()
  with tf.Graph().as_default():
    model = SoftmaxModel(config)
  
    # Create a session for running Ops on the Graph.
    sess = tf.Session()
  
    # Run the Op to initialize the variables.
    #init = tf.initialize_all_variables()()
    init = tf.global_variables_initializer()
    sess.run(init)
  
    losses = model.fit(sess, model.input_data, model.input_labels)

  # If ops are implemented correctly, the average loss should fall close to zero
  # rapidly.
  assert losses[-1] < .5
  print("Basic (non-exhaustive) classifier tests pass\n")

if __name__ == "__main__":
    test_SoftmaxModel()

Epoch 0: loss = 591.00 (0.051 sec)
Epoch 1: loss = 0.34 (0.023 sec)
Epoch 2: loss = 0.34 (0.027 sec)
Epoch 3: loss = 0.33 (0.027 sec)
Epoch 4: loss = 0.33 (0.023 sec)
Epoch 5: loss = 0.32 (0.023 sec)
Epoch 6: loss = 0.32 (0.021 sec)
Epoch 7: loss = 0.31 (0.020 sec)
Epoch 8: loss = 0.31 (0.021 sec)
Epoch 9: loss = 0.30 (0.022 sec)
Epoch 10: loss = 0.30 (0.024 sec)
Epoch 11: loss = 0.30 (0.029 sec)
Epoch 12: loss = 0.29 (0.024 sec)
Epoch 13: loss = 0.29 (0.022 sec)
Epoch 14: loss = 0.28 (0.021 sec)
Epoch 15: loss = 0.28 (0.021 sec)
Epoch 16: loss = 0.28 (0.020 sec)
Epoch 17: loss = 0.27 (0.020 sec)
Epoch 18: loss = 0.27 (0.021 sec)
Epoch 19: loss = 0.27 (0.023 sec)
Epoch 20: loss = 0.26 (0.030 sec)
Epoch 21: loss = 0.26 (0.023 sec)
Epoch 22: loss = 0.26 (0.023 sec)
Epoch 23: loss = 0.25 (0.020 sec)
Epoch 24: loss = 0.25 (0.020 sec)
Epoch 25: loss = 0.25 (0.029 sec)
Epoch 26: loss = 0.25 (0.032 sec)
Epoch 27: loss = 0.24 (0.023 sec)
Epoch 28: loss = 0.24 (0.026 sec)
Epoch 29: loss = 0.24 